In [1]:
import pandas as pd
import pandas

import matplotlib.pyplot as plt

import os

import numpy as np
import numpy
np.random.seed(42)

import seaborn as sns
import seaborn

import sklearn
import sklearn as sk

import copy
from collections import OrderedDict
%matplotlib inline

import seaborn
import seaborn as sns

In [2]:
data_classes = pd.read_csv('processed_data/data.csv')
data_static = pd.read_csv('raw_data/Static.csv', ';').drop('CLIENT_ID', axis=1)
data_join = pd.merge(data_static, data_classes, on='CONTRACT_ID')

In [3]:
contracts = np.array(data_static['CONTRACT_ID'].tolist())
shuffled  = copy.deepcopy(contracts)

length    = contracts.shape[0]
RATIO     = 0.1

test_contracts  = shuffled[:int(length * RATIO)].tolist()
train_contracts = shuffled[int(length * RATIO):].tolist()

data_learn = data_join.dropna(axis=0)

data_learn['GENDER'] = 1 * np.array(data_learn['GENDER'] == 'M')

data_train = data_learn[data_learn['CONTRACT_ID'].isin(train_contracts)].drop('CONTRACT_ID', axis=1)
data_test  = data_learn[data_learn['CONTRACT_ID'].isin( test_contracts)].drop('CONTRACT_ID', axis=1)

In [4]:
ways = ['sdtw_km', 'dba_km', 'km', 'gak_km', 'ks']
data_feats = {
    'train' : data_train.drop(columns=ways),
    'test'  :  data_test.drop(columns=ways)
}
data_classes = {
    'train' : {},
    'test'  : {}
}
for item in ways:
    data_classes['train'][item] = data_train[item]
    data_classes['test'][item] = data_test[item]

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score

rf_algos = {}
rf_scores = {}
rf_predictions = {}

for way in ways:
    clf_rf = RandomForestClassifier(n_estimators=300, max_depth=40, random_state=42, min_samples_leaf=10)
    clf_rf.fit(data_feats['train'], data_classes['train'][way])
    rf_predictions[way] = clf_rf.predict(data_feats['test'])
    rf_algos[way]  = clf_rf
    rf_scores[way] = {
        'accuracy' : accuracy_score(data_classes['test'][way], rf_predictions[way]),
        'f1_score' : f1_score(data_classes['test'][way], rf_predictions[way], average='weighted')
    }

/Users/work/Library/Python/3.7/lib/python/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/work/Library/Python/3.7/lib/python/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/work/Library/Python/3.7/lib/python/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/work/Library/Python/3.7/lib/python/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/w

In [14]:
for key, report in rf_scores.items():
    print(key, '\n', report)

sdtw_km 
 {'accuracy': 0.22691292875989447, 'f1_score': 0.19385874937598047}
dba_km 
 {'accuracy': 0.24538258575197888, 'f1_score': 0.1995088417122015}
km 
 {'accuracy': 0.5514511873350924, 'f1_score': 0.481848180950691}
gak_km 
 {'accuracy': 0.2691292875989446, 'f1_score': 0.233878617551794}
ks 
 {'accuracy': 0.33773087071240104, 'f1_score': 0.27101026418695023}


In [18]:
import pickle
pickle.dump(rf_algos, open('pickle_data/rf_clf.pickle', 'wb'))